# Preprocessing

In [1]:
import os
import re
from nltk.tokenize import word_tokenize
import numpy as np
import pickle
#from gensim.models import Word2Vec

In [2]:
diacritics_dict = {'َ': 0, 'ً': 1, 'ُ': 2, 'ٌ': 3, 'ِ': 4, 'ٍ': 5, 'ْ': 6, 'ّ': 7, 'َّ': 8, 'ًّ': 9, 'ُّ': 10, 'ٌّ': 11, 'ِّ': 12, 'ٍّ': 13, '': 14}

In [3]:
# Define a list of allowed characters (Arabic characters with diacritics)
allowed_characters = [
     'ذ', 'ت', 'ع', 'س', 'خ', 'ئ', 'ط', 'ث', 'ص', 'ح', 'إ', 'غ', 'ا', 'ب', 'ج', 'ز', 'آ', 'ر',
     'ش', 'ي', 'ض', 'ه', 'ق', 'ء', 'ن', 'د', 'ة', 'ف', 'ؤ', 'ظ', 'ل', 'م', 'ك', 'ى', 'أ', 'و', 
     'ّ', 'َ', 'ً', 'ٍ', 'ْ', 'ِ', 'ُ', 'ٌ'
 ]
def clean_text(text):
    
    # Remove characters not in the allowed list
    cleaned_text = ''.join(char if char in allowed_characters else ' ' for char in text)
    
    # Collapse consecutive spaces into a single space
    cleaned_text = ' '.join(cleaned_text.split())

    return cleaned_text

In [4]:
def tokenize_text(text):
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    return tokens

In [5]:
def preprocess_text(text):
    
    # Clean the text
    cleaned_text = clean_text(text)

    # Tokenize the cleaned text
    tokens = tokenize_text(cleaned_text)

    return tokens

In [6]:
# Extract letters in X and corresponding Diacritics in Y and same for words
def extract_arabic_and_Diacritics(arabic_tokens):
    # List of characters without Diacritics
    characters = []
    # List of words without Diacritics
    words = []
    # List of Diacritics corresponding to each character in characters list
    Diacritics_characters = []
    # List of Diacritics corresponding to each word in words list
    Diacritics_words = []

    # All Diacritics letters in Arabic
    Diacritics_set = ('َ', 'ُ', 'ِ', 'ّ', 'ْ', 'ٌ', 'ً', 'ٍ')

    # For each word with Diacritics
    for token in arabic_tokens:
        token_diacritic = []
        # Will be concatinated with letters without Diacritics
        word_without_Diacritics = ''
        # Check if the last letter in the word is Diacritics ==> Add it to Diacritics_words
        # Look at 2 letters as the diacritic may be 2 letters
        if token[-1] in Diacritics_set:
            token_diacritic.append(token[-1])
        # If the last character isn't Diacritics ==> Add None to Diacritics_words
        else:
            token_diacritic.append(None)
        if(len(token)>=2):
            if token[-2] in Diacritics_set:
                token_diacritic.append(token[-2])
            # If the last character isn't Diacritics ==> Add None to Diacritics_words
            else:
                token_diacritic.append(None)
        Diacritics_words.append(token_diacritic)
        # Loop over all characters of this word
        for char in token:
            # Check it isn't Diacritics (It is Arabic characher)
            if char not in Diacritics_set:
                # Add the character without Diacritics to characters list
                characters.append(char)
                # Concatinate the character without Diacritics to the word
                word_without_Diacritics += char

                # Extract Diacritics (it's the character after the Arabic letter)
                Diacritics_index = token.index(char) + 1
                # A list because it can be more than one Diacritics characters
                Diacritics = []
                # Check it is a Diacritics character
                if Diacritics_index < len(token) and token[Diacritics_index] in Diacritics_set:
                    Diacritics.append(token[Diacritics_index])
                    Diacritics_index +=1
                    # Diacritics may be 2 characters if it is (شدة + فتحة/ضمة/كسرة)
                    # Check for the character after the Diacritics character
                    if Diacritics_index < len(token) and token[Diacritics_index] in Diacritics_set:
                        Diacritics.append(token[Diacritics_index])
                    else:
                        # The second Diacritics character is None
                        Diacritics.append(None)
                else:
                    # No Diacritics
                    Diacritics.extend([None, None])
                # Add the Diacritics of the character to the list item corresponding to the character in characters list
                Diacritics_characters.append(Diacritics)
        # Add the word whithout Diacritics to the words list
        words.append(word_without_Diacritics)
        
                
    return characters, Diacritics_characters, words, Diacritics_words

In [7]:
def read_dataset(dataset_path):
    #dataset_path = r'/home/donia/Desktop/college/NLP/project/NLP-Arabic-Diacritics-Detector/dataset/train.txt'
    # Get the path to the current script's directory
    current_file_path = os.getcwd()
    
    # Construct the full path to the image
    dataset_path = os.path.join(current_file_path, dataset_path)
    
    with open(dataset_path, 'r', encoding='utf-8') as file:
        lines = [line.strip() for line in file.readlines()]
    
    # Split each line on dots to minimize length of line
    split_lines = [line.split('.') for line in lines]
    # Flatten the lines after splitting so each splitted part be a separate line
    # Remove empty lines
    flat_split_lines = [part for line in split_lines for part in line if part.strip()]
    
    #Lists to contain all the training data and its labels
    X_letters_list =[]
    Y_letters_list =[]
    X_words_list =[]
    Y_words_list =[]
    
    
    tokenized_lines =[]
    
    
    # Test the functions on the first 10 lines
    for line in flat_split_lines:
        cleaned_line = clean_text(line)
        removed = set(line) - set(cleaned_line)
        tokenized_line = tokenize_text(cleaned_line)
        
        X_letters, Y_letters, X_words, Y_words = extract_arabic_and_Diacritics(tokenized_line)
        X_letters_list.append(X_letters)
        Y_letters_list.append(Y_letters)
        X_words_list.append(X_words)
        Y_words_list.append(Y_words)
        tokenized_lines.append(tokenized_line)

    return X_letters_list, Y_letters_list, X_words_list, Y_words_list

In [8]:
def extract_letters_classes(Y_letters_list):
    letters_diacritics_classes = []
    for line in Y_letters_list:
        line_letters_diacritics_classes=[]
        for diacritic in line:
            diacritic_string = ""
            if(diacritic[0]):
                diacritic_string = diacritic[0]
            if(diacritic[1]):
                diacritic_string+=diacritic[1]
            line_letters_diacritics_classes.append(diacritics_dict[diacritic_string])
        letters_diacritics_classes.append(line_letters_diacritics_classes)
    return letters_diacritics_classes

In [9]:
# preprocessing training dataset
X_letters_list, Y_letters_list, X_words_list, Y_words_list = read_dataset("dataset/train.txt")

In [10]:
letters_diacritics_classes = extract_letters_classes(Y_letters_list)

In [11]:
# preprocessing validation dataset
val_X_letters_list, val_Y_letters_list, val_X_words_list, val_Y_words_list = read_dataset("dataset/val.txt")

In [12]:
val_letters_diacritics_classes = extract_letters_classes(val_Y_letters_list)

# One hot encoding

In [18]:
def generate_id_mapping(characters):
    char_to_id = {char: idx for idx, char in enumerate(characters)}
    id_to_char = {idx: char for idx, char in enumerate(characters)}
    return char_to_id, id_to_char

def generate_char_one_hot_vector(char_id, char_vector_size=45):
    char_vector = np.zeros(char_vector_size)
    char_vector[char_id] = 1
    return char_vector

def one_hot_encode_chars(char, char_to_id, char_vector_size=45):
    char_id = char_to_id.get(char, None)

    if char_id is not None:
        return generate_char_one_hot_vector(char_id, char_vector_size)
    else:
        return None

In [ ]:
characters = set([char for sequence in X_letters_list for char in sequence])

char_to_id, _ = generate_id_mapping(list(characters))

Generate one hot encoding for train dataset

In [26]:
characters = [char for sequence in X_letters_list for char in sequence]

X = [generate_char_one_hot_vector(char_to_id[char]) for char in characters ]

Y = [item for sublist in letters_diacritics_classes for item in sublist]

Generate one hot encoding for validation dataset

In [ ]:
characters = [char for sequence in val_X_letters_list for char in sequence]

val_X = [generate_char_one_hot_vector(char_to_id[char]) for char in characters ]

val_Y = [item for sublist in val_letters_diacritics_classes for item in sublist]